In [1]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from database.weather import Weather
from transformer.date_transformer import DateTransformer
from transformer.column_transformer import ColumnTransformer
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math
import numpy as np

In [2]:
## Loading Constants
start = "2008-01-01"
end = datetime(2021,1,7).strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("unity")
market = Market()
weather = Weather()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
correlations = market.retrieve_data("coint")
market.close()
reload = True
quarterly_range = range(1,5)
yearly_range = range(2018,2021)
dataset = "pdr"

In [3]:
market.connect()
classification = market.retrieve_data("dataset_pdr_week_classification")
regression = market.retrieve_data("dataset_pdr_week_regression")
accurate = market.retrieve_data("accurate")
relevant_tickers = list(accurate["ticker"])
relevant_tickers.extend(["date","year","quarter","week"])
market.close()
for col in regression.columns:
    if -99999 == regression[col].min() or col not in relevant_tickers:
        regression.drop(col,axis=1,inplace=True)
for col in classification.columns:
    if -99999 == classification[col].min() or col not in relevant_tickers:
        classification.drop(col,axis=1,inplace=True)

In [4]:
sims = []
gap = 5
week_gap = int(gap/5)
training_years = 7
timeline = DateUtils.create_timeline(start,end)
sims = []
sec.connect()
strat_db.connect()
market.connect()
market.drop_table("pdr_weekly_sim")
for year in tqdm(yearly_range):
    for quarter in tqdm(quarterly_range):
        try:
            for ticker in list(accurate["ticker"]):
                try:
                    if ticker in regression.columns:
                        price = market.retrieve_price_data("{}_prices".format(dataset),ticker)
                        if dataset == "pdr":
                            price = ColumnTransformer.rename_columns(price," ")
                        else:
                            price = ColumnTransformer.rename_columns(price,"")
                        price = DateTransformer.convert_to_date(dataset,price,"date")
                        mr = ModelPreprocessor(ticker)
                        prot = ProductTransformer(ticker,start,end)
                        ticker_regression = regression
                        ## regression_model
                        first = ticker_regression[(ticker_regression["year"] == year - training_years) & (ticker_regression["quarter"] == quarter)].index.values.tolist()[0]
                        last = ticker_regression[(ticker_regression["year"] == year) & (ticker_regression["quarter"] == quarter)].index.values.tolist()[0]
                        rqpd = ticker_regression.iloc[first:last-1]
                        rqpd["y"] = rqpd[ticker]
                        rqpd["y"] = rqpd["y"].shift(-week_gap)
                        rqpd = rqpd[:-week_gap]
                        qpd = mr.day_trade_preprocess_regression(rqpd.copy(),ticker,True)
                        rpr = sp.regression(qpd,ranked=False,tf=False,deep=False)
                        ## classification_model
                        ticker_classification = classification
                        first = ticker_classification[(ticker_classification["year"] == year - training_years) & (ticker_classification["quarter"] == quarter)].index.values.tolist()[0]
                        last = ticker_classification[(ticker_classification["year"] == year) & (ticker_classification["quarter"] == quarter)].index.values.tolist()[0]
                        cqpd = ticker_classification.iloc[first:last-1]
                        cqpd["y"] = cqpd[ticker]
                        cqpd["y"] = cqpd["y"].shift(-week_gap)
                        cqpd = cqpd[:-week_gap]
                        qpd = mr.day_trade_preprocess_classify(cqpd.copy(),ticker)
                        q2c = qpd["X"].columns
                        cpr = sp.classification(qpd,tf=False,deep=False)
                        price_results = pd.DataFrame([cpr,rpr])
                        product_qpds = []
                        current_sets = []
                        for j in range(len(price_results)):
                            price_result = price_results.iloc[j]
                            if price_result["model_type"] == "regression":
                                weekly_price_data = ticker_regression
                                weekly_price_data = weekly_price_data[(weekly_price_data["year"] == year) & (weekly_price_data["quarter"] == quarter)]
                                weekly_price_data["y"] = weekly_price_data[ticker]
                                product_qpd = mr.day_trade_preprocess_regression(weekly_price_data.copy(),ticker,False)
                            else:
                                weekly_price_data = ticker_classification
                                weekly_price_data = weekly_price_data[(weekly_price_data["year"] == year) & (weekly_price_data["quarter"] == quarter)]
                                weekly_price_data["y"] = weekly_price_data[ticker]
                                product_qpd = mr.day_trade_preprocess_classify(weekly_price_data.copy(),ticker)
                            price_dict = []
                            for week in product_qpd["X"]["week"]:
                                price_dict.append({"year":year,"quarter":quarter,"week":week})
                            price = pd.DataFrame(price_dict)
                            price_model = price_result["model"]
                            prediction = price_model.predict(product_qpd["X"])
                            current_set = price.copy()
                            col_name = price_result["model_type"]
                            current_set["weekly_{}_{}_prediction".format("price",col_name)] = prediction
                            current_set["weekly_{}_{}_score".format("price",col_name)] = price_result["score"]
                            current_sets.append(current_set)
                        base = current_sets[0]
                        for cs in current_sets[1:]:
                            base = base.merge(cs,on=["year","quarter","week"],how="left")
                        base["ticker"] = ticker
                        if len(base) > 0:
                            strat_db.store_data("{}_weekly_sim".format(dataset),base)
                except Exception as e:
                    message = {"status":"weekly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
                    print(message)
        except Exception as e:
            print(year,week,str(e))
sec.close()
market.close()
strat_db.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [16:33<00:00, 331.16s/it]
